Configurando o PyGithub

In [2]:
from github import Github

g = Github("token")

Buscando repositórios com mais de 5 mil estrelas de uma determinada linguagem

In [99]:
lang = 'javascript'

In [100]:
result = g.search_repositories(query="language:{0} stars:>5000 is:public".format(lang))

Métodos para acessar a API do code climate

In [101]:
import requests

def get_repo_id(full_name):
    try:
        resp = requests.get('https://api.codeclimate.com/v1/repos?github_slug={0}'.format(full_name))
        return resp.json()['data'][0]['id']
    except:
        return None 

def get_test_coverage(full_name):
    try:
        repo_id = get_repo_id(full_name)
        if repo_id is None:
            return None
        resp = requests.get('https://api.codeclimate.com/v1/repos/{0}/test_reports'.format(repo_id))
        return resp.json()['data'][0]['attributes']['covered_percent']
    except:
        return None

# print(get_test_coverage('rubocop-hq/rubocop'))

Itera sobre os repositórios e busca a cobertura de código

In [ ]:
repos = []
for r in result:
    repo = {
        'id': r.id,
        'name': r.name,
        'full_name': r.full_name,
        'description': r.description,
        'stargazers_count': r.stargazers_count,
        'open_issues_count': r.open_issues_count,
        'language': r.language,
        'test_coverage': get_test_coverage(r.full_name)
    }
    repos.append(repo)

In [ ]:
import pandas as pd

df = pd.DataFrame(repos)
df

Total de repositórios encontrados com mais de 5 mil estrelas

In [ ]:
len(repos)

Total de repositórios com cobertura reportados pelo Code Climate

In [ ]:
len(df[df['test_coverage'].notnull()])

Salvando em CSV

In [ ]:
df.to_csv('repos-code-climate-{0}.csv'.format(lang))